现在，我们已经有了一个完全匹配的数据[Full_Relevant_News](Full_Relevant_News.xlsx)和通过LAC以及thefuzz筛选后的数据[Cleaned_Other_news_with_ORGs_and_Similarities](Cleaned_Other_news_with_ORGs_and_Similarities.xlsx)。
现在，我们将这两个数据集合并，得到一个完整的数据集，来进行情感分析。
